In [ ]:
# export
from fastai_local.core import *
from fastai_local.test import *
from fastai_local.imports import *
from fastai_local.notebook.export import *
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor, Preprocessor
from nbconvert import HTMLExporter
from nbformat.sign import NotebookNotary
from traitlets.config import Config

In [ ]:
# default_exp notebook.export2html

# Converting notebooks to html

> The functions that transform the dev notebooks in the documentation of the library

## Preprocessing notebook

### Cell processors

In [ ]:
#export
def remove_widget_state(cell):
    "Remove widgets in the output of `cells`"
    if cell['cell_type'] == 'code' and 'outputs' in cell:
        cell['outputs'] = [l for l in cell['outputs'] 
                           if not ('data' in l and 'application/vnd.jupyter.widget-view+json' in l.data)]
    return cell

In [ ]:
#export
def hide_cells(cell):
    "Hide cell that need to be hidden"
    for pat in [r'^\s*#\s*export\s+', r'^\s*#\s*hide\s+', r'^\s*#\s*default_exp\s+', r's*show_doc\(']:
        if check_re(cell, pat): cell['metadata'] = {'hide_input': True}
    return cell

In [ ]:
for source in ['# export\nfrom fastai_local.core import *', 
               '# hide\nfrom fastai_local.core import *',
               '#default_exp notebook.export',
               'show_doc(read_nb)']:
    cell = {'cell_type': 'code', 'source': source}
    cell1 = hide_cells(cell.copy())
    assert 'metadata' in cell1
    assert 'hide_input' in cell1['metadata']
    assert cell1['metadata']['hide_input']

cell = {'cell_type': 'code', 'source': '# exports\nfrom fastai_local.core import *'}
test_eq(cell, hide_cells(cell.copy()))

In [ ]:
#export
def _show_doc_cell(name):
    return {'cell_type': 'code',
            'execution_count': None,
            'metadata': {},
            'outputs': [],
            'source': f"show_doc({name})"}

def add_show_docs(cells):
    "Add `show_doc` for each exported function or class"
    res = []
    for cell in cells:
        res.append(cell)
        if check_re(cell, r'^\s*#\s*exports?\s*'):
            names = func_class_names(cell['source'])
            for n in names: res.append(_show_doc_cell(n))
    return res

In [ ]:
tst_nb = read_nb('99_export.ipynb')
for i,cell in enumerate(tst_nb['cells']):
    if cell['source'].startswith('#export\ndef read_nb'): break
tst_cells = [c.copy() for c in tst_nb['cells'][i-1:i+1]]
added_cells = add_show_docs(tst_cells)
test_eq(len(added_cells), 3)
test_eq(added_cells[0], tst_nb['cells'][i-1])
test_eq(added_cells[1], tst_nb['cells'][i])
test_eq(added_cells[2], _show_doc_cell('read_nb'))

### Grabbing metada

In [ ]:
def get_metadata(cells):
    "Find the cell with title and summary in `cells`."
    pat = re.compile('^\s*#\s*([^\n]*)\n*>\s*([^\n]*)')
    for cell in cells:
        if cell['cell_type'] == 'markdown':
            match = re.match(pat, cell['source'])
            if match: return {'keywords': 'fastai',
                              'summary' : match.groups()[1],
                              'title'   : match.groups()[0]}

In [ ]:
test_eq(get_metadata(tst_nb['cells']), {
    'keywords': 'fastai',
    'summary': 'The functions that transform the dev notebooks in the fastai library',
    'title': 'Converting notebooks to modules'})

### Executing show_doc cells

In [ ]:
IMPORT_RE = re.compile(r"from (fastai_source[\.\w_]*)")
SHOW_DOC_RE = re.compile(r"show_doc\(([\w\.]*)")

In [ ]:
class ExecuteShowDocPreprocessor(ExecutePreprocessor):
    "An `ExecutePreprocessor` that only executes `show_doc` and `import` cells"
    def preprocess_cell(self, cell, resources, index):
        if 'source' in cell and cell.cell_type == "code":
            if IMPORT_RE.search(cell['source']) or SHOW_DOC_RE.search(cell['source']):
                return super().preprocess_cell(cell, resources, index)
        return cell, resources

In [ ]:
def execute_nb(nb, metadata=None, show_doc_only=True):
    "Execute `nb` (or only the `show_doc` cells) with `metadata`"
    ep_cls = ExecuteShowDocPreprocessor if show_doc_only else ExecutePreprocessor
    ep = ep_cls(timeout=600, kernel_name='python3')
    metadata = metadata or {}
    ep.preprocess(nb, metadata)

In [ ]:
fake_nb = {k:v for k,v in tst_nb.items() if k != 'cells'}
fake_nb['cells'] = tst_nb['cells'][0].copy() + added_cells


ValueError: dictionary update sequence element #0 has length 3; 2 is required

In [ ]:
fake_nb

{'metadata': {'kernelspec': {'display_name': 'Python 3',
   'language': 'python',
   'name': 'python3'},
  'language_info': {'codemirror_mode': {'name': 'ipython', 'version': 3},
   'file_extension': '.py',
   'mimetype': 'text/x-python',
   'name': 'python',
   'nbconvert_exporter': 'python',
   'pygments_lexer': 'ipython3',
   'version': '3.7.1'}},
 'nbformat': 4,
 'nbformat_minor': 2,
 'cells': [{'cell_type': 'markdown',
   'metadata': {},
   'source': "A jupyter notebook is a json file behind the scenes. We can just read it with the json module, which will return a nested dictionary of dictionaries/lists of dictionaries, but there are some small differences between reading the json and using the tools from `nbformat` so we'll use this one."},
  {'cell_type': 'code',
   'execution_count': 3,
   'metadata': {},
   'outputs': [],
   'source': '#export\ndef read_nb(fname):\n    "Read the notebook in `fname`."\n    with open(Path(fname),\'r\') as f: return nbformat.reads(f.read(), as_ve

## Conversion

In [ ]:
class NbExporter(HTMLExporter):
    
    @property
    def template_path(self):
        return super().template_path+[str((Path('fastai_docs')/'notebook'/).absolute())]

    def _template_file_default(self):
        """
        We want to use the new template we ship with our library.
        """
        return 'test_template' # full

In [ ]:
def _exporter():
    exporter = HTMLExporter(Config())
    exporter.exclude_input_prompt=True
    exporter.exclude_output_prompt=True
    exporter.template_file = 'jekyll.tpl'
    exporter.template_path.append(str((Path('fastai_local')/'notebook').absolute()))
    return exporter

In [ ]:
def convert_nb(fname, dest_path='.'):
    "Convert a notebook `fname` to html file in `dest_path`."
    nb = read_nb(fname)
    nb['cells'] = [compose(*process_cells)(c) for c in nb['cells']]
    fname = Path(fname).absolute()
    dest_name = fname.with_suffix('.html').name.split('_')[-1]
    meta_jekyll = get_metadata(nb['cells'])
    meta_jekyll['nb_path'] = f'{fname.parent.name}/{fname.name}'
    with open(f'{dest_path}/{dest_name}','w') as f:
        f.write(_exporter().from_notebook_node(nb, resources=meta_jekyll)[0])

In [ ]:
convert_nb('99_export.ipynb')

In [ ]:
%debug

> /home/ubuntu/anaconda3/lib/python3.7/site-packages/jinja2/loaders.py(408)load()
    406             except TemplateNotFound:
    407                 pass
--> 408         raise TemplateNotFound(name)
    409 
    410     def list_templates(self):

ipdb> u
> /home/ubuntu/anaconda3/lib/python3.7/site-packages/jinja2/environment.py(804)_load_template()
    802                                          template.is_up_to_date):
    803                 return template
--> 804         template = self.loader.load(self, name, globals)
    805         if self.cache is not None:
    806             self.cache[cache_key] = template

ipdb> globals
{'range': <class 'range'>, 'dict': <class 'dict'>, 'lipsum': <function generate_lorem_ipsum at 0x7f7804220950>, 'cycler': <class 'jinja2.utils.Cycler'>, 'joiner': <class 'jinja2.utils.Joiner'>, 'namespace': <class 'jinja2.utils.Namespace'>, 'uuid4': <function uuid4 at 0x7f7877229158>}
ipdb> self
ipdb> name
'jekyll.tpl'
ipdb> u
> /home/ubuntu/anaconda3/lib

In [ ]:
(Path('/home/ubuntu/fastai_docs/dev/fastai_docs/notebook')/'jekyll.tpl').exists()

False